# Assignment 3 - Neural Volume Rendering and Surface Rendering

## A. Neural Volume Rendering

### 0. Transmittance Calculation:

<img src="images/part_0.png" alt="Grid Visualization" width="600">

### 1. Differentiable Volume Rendering:

#### 1.3 Ray Sampling

<table>
    <tr>
        <th>Grid Visualization</th>
        <th>Ray Visualization</th>
    </tr>
    <tr>
        <td><img src="images/1_3_grid_rays.png" alt="Grid Visualization" width="400"></td>
        <td><img src="images/1_3_grid_vis.png" alt="Ray Visualization" width="400"></td>
    </tr>
</table>

#### 1.4 Point Sampling 

<table>
    <tr>
        <th>Sampled Points</th>
    </tr>
    <tr>
        <td><img src="images/1_4.png" alt="Grid Visualization" width="400"></td>    
    </tr>
</table>

#### 1.5 Volume Rendering 

<table>
    <tr>
        <th>Visualization</th>
        <th>Depth Map</th>
    </tr>
    <tr>
        <td><img src="images/part_1.gif" alt="Grid Visualization" width="400"></td>
        <td><img src="images/1_5_depth_map.png" alt="Ray Visualization" width="400"></td>
    </tr>
</table>

### 2. Optimizing a basic implicit volume

#### 2.3 Visualization

Box Center: [0.25, 0.25, 0.0]  

Side Lengths: [2.01, 1.50, 1.50]

<table>
    <tr>
        <th>Optimized Volume</th>
        <th>TA Result</th>
    </tr>
    <tr>
        <td><img src="images/part_2.gif" alt="Grid Visualization" width="400"></td> 
        <td><img src="ta_images/part_2.gif" alt="Grid Visualization" width="400"></td>  
    </tr>
</table>

### 3. Optimizing a Neural Radiance Field (NeRF)

<table>
    <tr>
        <th>Lego Bulldozer</th>
    </tr>
    <tr>
        <td><img src="images/part_3_og.gif" alt="Grid Visualization" width="300"></td>  
    </tr>
</table>

### 4. NeRF Extras

#### 4.1 View Dependence

**Trade-offs:**

Adding view dependence allows NeRF to model effects like reflections and specular highlights by making color depend on the viewing direction. However, this increases the risk of overfitting to the training images, making it harder for the model to generalize to unseen viewpoints. A balance must be struck—too much view dependence can lead to memorization, while too little may fail to capture realistic lighting effects.

<table>
    <tr>
        <th>Lego Bulldozer</th>
        <th>Materials</th>
        <th>Materials High res</th>
    </tr>
    <tr>
        <td><img src="images/part_4_bulldozer.gif" alt="Grid Visualization" width="400"></td>
        <td><img src="images/part_4_materials.gif" alt="Grid Visualization" width="400"></td>
        <td><img src="images/part_4_material_highres.gif" alt="Grid Visualization"></td>  
    </tr>
</table>

#### 4.2 Coarse/Fine Sampling 

**Approach**

The ray sampling implementation is inspired by the [NeuS](https://arxiv.org/pdf/2106.10689) paper. The process involves:  
1. Sampling an initial set of points along the ray.  
2. Using an implicit function to compute densities at each sampled point.  
3. Resampling points along the ray based on these densities, ensuring more points are allocated to regions of higher density.  

Unlike the NeRF implementation, this approach uses only one model for both coarse and fine sampling.

**Trade-offs**

1. Speed: The fine/coarse implementation takes 18secs/epoch while without it, it takes 13secs/epoch. Thus, fine/coarse implementation is slower, which is expected due to having to pass the sampled points twice to the model.

2. Quality: For some reason, it gave a blank output, so cant really comment on the quality 


<table>
    <tr>
        <th>Without fine/coarse sampling</th>
        <th>With fine/coarse sampling</th>
    </tr>
    <tr>
        <td><img src="images/part_4_bulldozer.gif" alt="Grid Visualization" width="400"></td>
        <td><img src="images/part_4_2.gif" alt="Grid Visualization" width="400"></td>  
    </tr>
</table>

## B. Neural Surface Rendering

### 5. Surface Tracing

**Implementation:**
My implementation starts with the origins, calculates shortest distances for each point using implicit function and marches along the ray by this distance. This process is repeated for each point for 'max_iter' (250) times. And mask is maintained for points which have distance < eps (a very small value). 

<table>
    <tr>
        <th>Torus</th>
    </tr>
    <tr>
        <td><img src="images/part_5_torus.gif" alt="Grid Visualization" width="400"></td>
    </tr>
</table>

### 6. Optimizing a Neural SDF

**MLP Model:**

I am using almost the same model as used for volume rendering except for changing the relevant variables.
The model for predicting color and distance can be explained as:

1. input points are embedded to higher dimension
2. these are then passed by 5 linear layers and relu activation
3. the output is concatenated with embedded input points (skip connection) to give x
4. x is passed through 4 linear layers, each followed by relu except last layer to output the distance. 
5. After step 3, x is separately passed through another 3 linear layers (relu activation, except sigmoid activation for last layer) to predict color

**Eikonal Loss:**

To calculate Eikonal Loss, first norm of gradient is calculated and then MSE of this norm is applied against array of ones to penalize deviations from 1.

<table>
    <tr>
        <th>Input</th>
        <th>Predicted</th>
    </tr>
    <tr>
        <td><img src="images/part_6_input.gif" alt="Grid Visualization" width="400"></td>
        <td><img src="images/part_6.gif" alt="Grid Visualization" width="400"></td>  
    </tr>
</table>


### 7. VolSDF

**beta:** Controls sharpness of the surface. Higher beta generates smoother transition and thus blurry surface while lower beta changes density more abruptly.

**alpha:** Is a scaling factor that increases or decreases the overall density. Thus, higher alpha makes the surface more opaque while lower alpha give transparent effect.

1. **How does high beta bias you learned SDF? What about low beta?** 
- High beta makes the density function more spread out, thus SDF transitions are smoother. This results in blurry surface representation. 
- While low beta makes density function sharper and we get sharper boundaries, it can make training less stable.

2. **Would an SDF be easier to train with volume rendering and low beta or high beta? Why?**
- It would be easier to train with high beta because smooth transitions help gradients propagate effectively during training, while low beta changes density rapidly, making optimization harder and leading to unstable updates.

3. **Would you be more likely to learn an accurate surface with high beta or low beta? Why?**
- Low beta is likely to learn accurate surface becuase it can learn sharper boundaries, thus capturing precise details.


<table>
    <tr>
        <th>Geometry</th>
        <th>Best output</th>
    </tr>
    <tr>
        <td><img src="images/part_7_geometry.gif" alt="Grid Visualization" width="300"></td>
        <td><img src="images/part_7_normal_a_20_b_0_05.gif" alt="Grid Visualization" width="300"></td>  
    </tr>
</table>

As discussed before, higher alpha gives more opaque surface, as seen between test 1 and test 2. While increasing beta gives blurrier output (test 2 -> test 3). Thus test 2 gave the best output among these.

<table>
    <tr>
        <th>Test conditions</th>
        <th>alpha = 10; beta = 0.05</th>
        <th> (Best) alpha = 20; beta = 0.05 </th>
        <th>alpha = 20; beta = 0.5</th >
    </tr>
    <tr>
        <th>Outputs</th>
        <td><img src="images/part_7_normal_a_10_b_0.05.gif" alt="Grid Visualization" width="300"></td>
        <td><img src="images/part_7_normal_a_20_b_0_05.gif" alt="Grid Visualization" width="300"></td>
        <td><img src="images/part_7_normal_a_20_0_5.gif" alt="Grid Visualization" width="300"></td>   
    </tr>
</table>

### 8. Neural Surface Extras

#### 8.1 Render a large scene with sphere tracing 

<table>
    <tr>
        <th>Scene with 4 torus in multiple locations and shapes</th>
    </tr>
    <tr>
        <td><img src="images/part_8_1_multipletorus.gif" alt="Grid Visualization" width="400"></td> 
    </tr>
</table>

#### 8.2 Fewer training views

Volume and Surface training is done using 20, 50 and 100 views.

<table>
    <tr>
        <th>Number of views</th>
        <th>20</th>
        <th>50</th>
        <th>100</th>
    </tr>
    <tr>
        <td>Volume Rendering</td>
        <td><img src="images/part_4_normal_20views.gif" alt="Grid Visualization" width="250"></td>
        <td><img src="images/part_4_normal_50views.gif" alt="Grid Visualization" width="250"></td> 
        <td><img src="images/part_4_bulldozer.gif" alt="Grid Visualization" width="250"></td>
    </tr>
    <tr>
        <td>Surface Rendering</td>
        <td><img src="images/part_7_naive_20views.gif" alt="Grid Visualization" width="250"></td>
        <td><img src="images/part_7_normal_50_views.gif" alt="Grid Visualization" width="250"></td> 
        <td><img src="images/part_7_normal_a_20_b_0_05.gif" alt="Grid Visualization" width="250"></td>
    </tr>
</table>

#### 8.3 Alternate SDF to Density Conversions

I implemented Naive gaussian conversion method to convert sdf to density based on the function:

$$
\rho(s, \beta) = \exp\left(-\frac{s^2}{2\beta^2}\right)
$$

<table>
    <tr>
        <th>SDF 2 Density </th>
        <th>Naive Guassian Conversion (beta = 0.5)</th>
    </tr>
    <tr>
        <td><img src="images/part_7_normal_a_20_b_0_05.gif" alt="Grid Visualization" width="250"></td>
        <td><img src="images/part_7_naive_beta_0.5.gif" alt="Grid Visualization" width="250"></td>
    </tr>
</table>